In [1]:
import tensorflow as tf
import os

C:\Users\sunqu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICE"] = "0"

In [3]:
def weight(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
W1=5;H1=5;C1=32
W2=5;H2=5;C2=64
C3=1024

In [7]:
with tf.name_scope('input_data'):
    x = tf.placeholder(tf.float32, [None,28*28])
    y_gt = tf.placeholder(tf.float32, [None,10])
    x_image = tf.reshape(x, [-1,28,28,1])
    tf.summary.image('input', x_image, max_outputs=10)

In [8]:
with tf.name_scope('hidden1'):
    W_conv1 = weight([W1,H1,1,C1], name='weights')
    b_conv1 = bias([C1], name='bias')
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    tf.summary.histogram('histogram of W1', W_conv1)
    tf.summary.histogram('histogram of b1', b_conv1)

INFO:tensorflow:Summary name histogram of W1 is illegal; using histogram_of_W1 instead.
INFO:tensorflow:Summary name histogram of b1 is illegal; using histogram_of_b1 instead.


In [10]:
with tf.name_scope('hidden2'):
    W_conv2 = weight([W2,H2,C1,C2], name='weights')
    b_conv2 = weight([C2], name='bias')
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    tf.summary.histogram('histogram of W2', W_conv2)
    tf.summary.histogram('histogram of b2', b_conv2)

INFO:tensorflow:Summary name histogram of W2 is illegal; using histogram_of_W2 instead.
INFO:tensorflow:Summary name histogram of b2 is illegal; using histogram_of_b2 instead.


In [11]:
with tf.name_scope('fully_conneted'):
    W_fc = weight([7*7*C2, C3], name='weights')
    b_fc = bias([C3], name='bias')
    h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*C2])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc) + b_fc)
    tf.summary.histogram('hsitogram of W_fc', W_fc)
    tf.summary.histogram('histogram of b_fc', b_fc)

INFO:tensorflow:Summary name hsitogram of W_fc is illegal; using hsitogram_of_W_fc instead.
INFO:tensorflow:Summary name histogram of b_fc is illegal; using histogram_of_b_fc instead.


In [12]:
with tf.name_scope('dropout'):
    p_keep = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc, p_keep)

In [15]:
with tf.name_scope('output_layer'):
    W_out = weight([C3,10], name='weights')
    b_out = bias([10], name='bias')
    y_pred = tf.matmul(h_fc_drop, W_out) + b_out
    tf.summary.histogram('histogram of W_out', W_out)
    tf.summary.histogram('histogram of b_out', b_out)

INFO:tensorflow:Summary name histogram of W_out is illegal; using histogram_of_W_out instead.
INFO:tensorflow:Summary name histogram of b_out is illegal; using histogram_of_b_out instead.


In [16]:
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_gt, logits=y_pred))
    tf.summary.scalar('loss of model', cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Summary name loss of model is illegal; using loss_of_model instead.


In [17]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entropy)

In [19]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_gt,1), tf.argmax(y_pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [20]:
epochs = 20000
batch_size = 64

In [ ]:
with tf.Session() as sess:
    summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('MNIST_logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('MNIST_logs/test', sess.graph)
    sess.run(tf.global_variables_initializer())
    
    for iter in range(epochs):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x:batch_x, y_gt:batch_y, p_keep:0.5})
        if iter % 100 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={x:batch_x, y_gt:batch_y, p_keep:1.0})
            train_accuracy = sess.run(accuracy, feed_dict={x:batch_x, y_gt:batch_y, p_keep:1.0})
            
            test_accuracy = sess.run(accuracy, feed_dict={x:mnist.test.images, y_gt:mnist.test.labels, p_keep:1.0})
            print('iter %d, train accuracy %f, test accuracy %f' % (iter, train_accuracy, test_accuracy))
            
            summary_train = sess.run(summary,{x:batch_x,y_gt:batch_y,p_keep:1.0})
            summary_test = sess.run(summary,{x:mnist.test.images,y_gt:mnist.test.labels,p_keep:1.0})
            train_writer.add_summary(summary_train, iter)
            test_writer.add_summary(summary_test, iter)
            train_writer.flush()
            test_writer.flush()
train_writer.close()
test_writer.close()

iter 0, train accuracy 0.062500, test accuracy 0.050100
iter 100, train accuracy 0.875000, test accuracy 0.878100
iter 200, train accuracy 0.906250, test accuracy 0.915700
iter 300, train accuracy 0.937500, test accuracy 0.933700
iter 400, train accuracy 0.968750, test accuracy 0.940200
iter 500, train accuracy 0.953125, test accuracy 0.950300
iter 600, train accuracy 0.906250, test accuracy 0.954700
iter 700, train accuracy 0.953125, test accuracy 0.959000
iter 800, train accuracy 1.000000, test accuracy 0.962500
iter 900, train accuracy 0.953125, test accuracy 0.964100
iter 1000, train accuracy 0.953125, test accuracy 0.964800
iter 1100, train accuracy 0.984375, test accuracy 0.968000
iter 1200, train accuracy 1.000000, test accuracy 0.970100
iter 1300, train accuracy 0.968750, test accuracy 0.973000
iter 1400, train accuracy 0.984375, test accuracy 0.973200
iter 1500, train accuracy 0.953125, test accuracy 0.972500
iter 1600, train accuracy 1.000000, test accuracy 0.975100
iter 1700

iter 13900, train accuracy 1.000000, test accuracy 0.991800
iter 14000, train accuracy 1.000000, test accuracy 0.992300
iter 14100, train accuracy 1.000000, test accuracy 0.992600
iter 14200, train accuracy 1.000000, test accuracy 0.991900
iter 14300, train accuracy 1.000000, test accuracy 0.991500
iter 14400, train accuracy 1.000000, test accuracy 0.991900


In [ ]:
tensorboard --logdir MNIST_logs